In [2]:
import argparse
from collections import defaultdict, namedtuple
from io import open
import math
import os
from random import shuffle, uniform

from get_data import load_data, extract_features
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding, Input, Concatenate


In [25]:
def lstm(train, test):
#     parser = argparse.ArgumentParser(description='Duolingo shared task baseline model')
#     parser.add_argument('--train', help='Training file name', required=True)
#     parser.add_argument('--test', help='Test file name, to make predictions on', required=True)
#     parser.add_argument('--pred', help='Output file name for predictions, defaults to test_name.pred')
#     args = parser.parse_args()

#     if not args.pred:
#         args.pred = args.test + '.pred'

#     assert os.path.isfile(args.train)
#     assert os.path.isfile(args.test)

#     # Assert that the train course matches the test course
#     assert os.path.basename(args.train)[:5] == os.path.basename(args.test)[:5]

    training_data, training_labels = load_data(train)
    test_data = load_data(test)
    return training_data, training_labels, test_data


In [26]:
training_data, training_labels, test_data = lstm("data_en_es/en_es.slam.20190204.train", "data_en_es/en_es.slam.20190204.test")

Loading instances...
Loaded 317049 instances across 100000 exercises...
Loaded 635368 instances across 200000 exercises...
Loaded 951536 instances across 300000 exercises...
Loaded 1271940 instances across 400000 exercises...
Loaded 1591344 instances across 500000 exercises...
Loaded 1911212 instances across 600000 exercises...
Loaded 2227444 instances across 700000 exercises...
Loaded 2546704 instances across 800000 exercises...
Done loading 2622957 instances across 824012 exercises.

Loading instances...
Loaded 337728 instances across 100000 exercises...
Done loading 386604 instances across 114586 exercises.



In [28]:
extract_features(training_data)

Total features: 4636
Building embedding matrix...


TypeError: Dimension value must be integer or None or have an __index__ method, got <tf.Tensor 'input_7:0' shape=(None, None) dtype=float32>